In [12]:
import pandas as pd

df_query_dev = pd.read_pickle('bm25_results_dev.pkl')

# Uploading the collection (corpus of articles)
import pickle

with open('subtask4b_collection_data.pkl', 'rb') as f:
    df_collection = pickle.load(f)


/var/folders/wl/s2d0f7l51w3gpcrn7gtnv0tc0000gn/T/ipykernel_46036/4286443179.py:9: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  df_collection = pickle.load(f)


In [14]:
df_query_dev.columns

Index(['post_id', 'tweet_text', 'cord_uid', 'bm25_topk'], dtype='object')

In [13]:
!pip install sentence-transformers


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip


In [15]:
from sentence_transformers import CrossEncoder

cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')


In [ ]:
def rerank(query_text, candidate_ids, collection_df):
    candidate_texts = []
    for uid in candidate_ids:
        row = collection_df[collection_df['cord_uid'] == uid]
        if not row.empty:
            title = row.iloc[0]['title'] if pd.notnull(row.iloc[0]['title']) else ''
            abstract = row.iloc[0]['abstract'] if pd.notnull(row.iloc[0]['abstract']) else ''
            doc_text = f"{title} {abstract}"
            candidate_texts.append(doc_text)
        else:
            candidate_texts.append('')  # if uid not found
    
    pairs = [[query_text, doc] for doc in candidate_texts]
    
    scores = cross_encoder.predict(pairs)
    
    # Sort by score
    indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:5]  # top-5
    reranked_uids = [candidate_ids[i] for i in indices]
    
    return reranked_uids


In [18]:
df_query_dev['reranked_top5'] = df_query_dev.apply(
    lambda row: rerank(row['tweet_text'], row['bm25_topk'], df_collection),
    axis=1
)


In [19]:
df_query_dev[['post_id', 'reranked_top5']].rename(columns={'reranked_top5': 'preds'}).to_csv('reranked_predictions.tsv', sep='\t', index=False)


In [20]:
df_query_dev

,post_id,tweet_text,cord_uid,bm25_topk,reranked_top5
0,16,covid recovery: this study from the usa reveal...,3qvh482o,"[25aj8rj5, gatxuwz7, 59up4v56, styavbvi, 6sy80...","[59up4v56, styavbvi, 25aj8rj5, gatxuwz7, 6sy80..."
1,69,"""Among 139 clients exposed to two symptomatic ...",r58aohnu,"[r58aohnu, p0kg6dyz, 9dlaaye8, iu1d9i57, d06np...","[r58aohnu, d06npvro, p0kg6dyz, 9dlaaye8, iu1d9..."
2,73,I recall early on reading that researchers who...,sts48u9i,"[tz2shoso, o877uul1, m1sf159a, sgo76prc, gruir...","[gruir7aw, o877uul1, tz2shoso, sgo76prc, m1sf1..."
3,93,You know you're credible when NIH website has ...,3sr2exq9,"[3sr2exq9, hgpiig0g, sv48gjkk, k0f4cwig, ihgxt...","[k0f4cwig, 3sr2exq9, sv48gjkk, hgpiig0g, ihgxt..."
4,96,Resistance to antifungal medications is a grow...,ybwwmyqy,"[lzddnb8j, ouvq2wpq, sxx3yid9, vabb2f26, y9fqa...","[lzddnb8j, vabb2f26, ouvq2wpq, sxx3yid9, y9fqa..."
...,...,...,...,...,...
1395,14193,Residents at high risk of covid-19: effectiven...,0gn3b98n,"[0gn3b98n, soa71kwx, 0e97tdez, feohcey3, pqjum...","[0gn3b98n, 0e97tdez, soa71kwx, feohcey3, pqjum..."
1396,14196,"61% of teenagers hospitalized for covid were ""...",25bdifv6,"[s1gdbsfx, 92z77upt, 5m23wtpi, j8evw0qj, xtmn1...","[j8evw0qj, 92z77upt, xtmn1n0r, 5m23wtpi, s1gdb..."
1397,14203,"""fresh evidence backing melatonin against covi...",qn6wawxk,"[qn6wawxk, dsz66r4u, m80rklg5, rtus1qyw, 059oj...","[qn6wawxk, dsz66r4u, rtus1qyw, 059oj76m, m80rk..."
1398,14233,"the vaccine doesn't halt the spread, it is pro...",3u3i5myh,"[unl3n0tn, 2u9eenwu, d6ow67k9, 582ld85p, 47p4g...","[unl3n0tn, 47p4g80o, 582ld85p, 2u9eenwu, d6ow6..."


In [22]:
def get_performance_mrr(data, col_gold, col_pred, list_k=[1, 5, 10]):
    d_performance = {}
    for k in list_k:
        data["in_topx"] = data.apply(
            lambda x: (1 / (x[col_pred][:k].index(x[col_gold]) + 1)
                       if x[col_gold] in x[col_pred][:k] else 0),
            axis=1
        )
        d_performance[k] = data["in_topx"].mean()
    return d_performance

In [23]:
results_reranked = get_performance_mrr(df_query_dev, 'cord_uid', 'reranked_top5')
print(f"Results after Neural Re-Ranking: {results_reranked}")


Results after Neural Re-Ranking: {1: np.float64(0.535), 5: np.float64(0.5738809523809524), 10: np.float64(0.5738809523809524)}


In [38]:
from rank_bm25 import BM25Okapi
import numpy as np


In [34]:
df_query_test = pd.read_csv('subtask4b_query_tweets_test.tsv', sep='\t')


In [35]:
corpus = df_collection[['title', 'abstract']].apply(lambda x: f"{x['title']} {x['abstract']}", axis=1).tolist()
cord_uids = df_collection['cord_uid'].tolist()

tokenized_corpus = [doc.split(' ') for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

In [36]:
text2bm25top = {}

def get_top_cord_uids(query):
  if query in text2bm25top.keys():
      return text2bm25top[query]
  else:
      tokenized_query = query.split(' ')
      doc_scores = bm25.get_scores(tokenized_query)
      indices = np.argsort(-doc_scores)[:5]
      bm25_topk = [cord_uids[x] for x in indices]

      text2bm25top[query] = bm25_topk
      return bm25_topk

In [39]:
df_query_test['bm25_topk'] = df_query_test['tweet_text'].apply(lambda x: get_top_cord_uids(x))


In [ ]:
df_query_test['reranked_top5'] = df_query_test.apply(
    lambda row: rerank(row['tweet_text'], row['bm25_topk'], df_collection),
    axis=1
)

# Saving results submission
df_query_test[['post_id', 'reranked_top5']].rename(columns={'reranked_top5': 'preds'}).to_csv(
    'reranked_predictions_test.tsv', sep='\t', index=False
)
